In [7]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808329 sha256=9e7a6675c62aff7fce66ba6e217db4ab553f72c7aacc6f75dd3148d0bf047a6f
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [8]:
# from lightfm.data import Dataset
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import requests
from collections import deque
from lightfm.data import Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Googlecolab/user_songs_filtered.csv')
users = pd.read_csv('/content/drive/MyDrive/Googlecolab/users_feature_eng.csv')
users.head()

Mounted at /content/drive


,Username,track_name,artist_name,rank,playcount,country,log_total_track_count,log_total_artist_count,activeness_most inactive,activeness_inactive,activeness_medium,activeness_active,activeness_most active
0,emosoup,Higher,Sleep Token,1,1321,United States,0.740725,0.686149,False,False,True,False,False
1,maiconslavieiro,Higher,Sleep Token,21,151,Brazil,0.766376,0.741361,False,False,False,True,False
2,velenious,Higher,Sleep Token,19,1259,United States,0.808702,0.766860,False,False,False,False,True
3,Antimemetic,Higher,Sleep Token,32,29,United States,0.646481,0.554484,True,False,False,False,False
4,frankcreature,Higher,Sleep Token,43,43,Czech Republic,0.610178,0.553889,True,False,False,False,False


In [3]:
songs = pd.read_csv('/content/drive/MyDrive/Googlecolab/songs_feature_eng_pca.csv')
songs.head()

,log_listeners,log_total_playcount,polarity,subjectivity,rms,spectral_centroid,zcr,tempo,mfcc_mean,mfcc_var,...,mfcc_19,mfcc_20,PC1,PC2,PC3,PC4,PC5,PC6,track_name,artist_name
0,-0.167387,0.023660,-0.165781,-0.036365,0.370126,0.184518,0.131208,0.188372,-0.099412,-0.258117,...,-0.475994,-0.437934,-1.448596,-0.569393,-0.016804,0.335455,-0.061101,0.057167,One Look,Leo
1,0.469054,0.392145,-1.317889,0.779821,-0.464388,0.252576,0.488321,1.088372,0.025586,0.022412,...,-0.510913,-0.384981,-2.433523,-0.022320,-0.594790,0.433389,1.437584,-0.308513,Cradle,Mudvayne
2,-0.221593,-0.191305,-0.659764,-0.850362,0.195520,0.466441,0.512684,1.212510,-0.018932,-0.367260,...,-0.570185,-0.548602,-1.074580,-1.403430,0.398831,0.141553,-0.098781,-1.219734,Reading YouTube Comments,Drug Church
3,0.363056,0.461095,-0.028703,0.399409,0.660554,0.452705,0.509389,-0.616891,-0.025497,-0.381531,...,-0.493999,-0.696913,0.301323,-1.488867,-0.539187,0.851189,0.265387,0.105254,Tissues,YUNGBLUD
4,0.789597,0.921555,0.181731,-0.219415,0.542149,0.464976,0.385042,0.761099,-0.057963,-0.369105,...,-0.437318,-0.663161,-1.504924,-1.040311,-1.218933,0.622659,-0.393141,-0.040788,The Games We Play,Pusha T


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Googlecolab/user_songs_filtered.csv')
df_top_tracks = df

# Group by 'Username' and aggregate the other columns into lists
aggregated_data = df_top_tracks.groupby('Username').agg({
    'track_name': list,
    'artist_name': list,
    'playcount': list
}).reset_index()


# Rename the columns to match the second image
aggregated_data.rename(columns={
    'Username': 'User',
    'track_name': 'Tracks',
    'artist_name': 'Artists',
    'playcount': 'Playcounts'
}, inplace=True)

# The resulting DataFrame should be in the desired format
aggregated_data.head()


,User,Tracks,Artists,Playcounts
0,-Dolorosa-,"[I Will Follow You Into The Dark, Destruction ...","[Death Cab for Cutie, Chelsea Wolfe, Have a Ni...","[283, 225, 187, 190, 263, 302, 174, 327, 173, ..."
1,-itssoeasy,"[Drive, Dig, Holding Someone's Hair Back, Anna...","[Incubus, Incubus, Circa Survive, Incubus, Kil...","[62, 42, 23, 45, 23, 17, 44, 20, 23, 25, 55, 1..."
2,0-172,"[Head Hunter, Pachuca Sunrise, Throwin' Shapes...","[Dance Gavin Dance, Minus the Bear, Minus the ...","[154, 185, 166, 144, 195, 165, 193, 185, 180, ..."
3,23linear,"[8000, Crystalline, Neurosomatic Circuit, Proc...","[Extrawelt, Younger Brother, Androcell, Androc...","[68, 97, 73, 77, 324, 262, 142, 120, 118, 112,..."
4,40belowsummer,"[My Own Summer (Shove It), One Step Closer, In...","[Deftones, Linkin Park, Linkin Park, Faith No ...","[166, 130, 155, 134, 152, 169, 131, 150, 142, ..."


In [9]:
records = []
for i, row in aggregated_data.iterrows():
    user = row['User']
    for track, artist, playcount in zip(row['Tracks'], row['Artists'], row['Playcounts']):
        track_artist = f"{track} - {artist}"
        records.append((user, track_artist, playcount))

df_flat = pd.DataFrame(records, columns=['User', 'Track_Artist', 'Playcount'])
print(df_flat)
dataset = Dataset()
dataset.fit(users=df_flat['User'].unique(),
            items=df_flat['Track_Artist'].unique())

(interactions, weights) = dataset.build_interactions([(x['User'], x['Track_Artist'], float(x['Playcount'])) for index, x in df_flat.iterrows()])

               User                                       Track_Artist  \
0        -Dolorosa-  I Will Follow You Into The Dark - Death Cab fo...   
1        -Dolorosa-  Destruction Makes the World Burn Brighter - Ch...   
2        -Dolorosa-                       Bloodhail - Have a Nice Life   
3        -Dolorosa-                      Such Small Hands - La Dispute   
4        -Dolorosa-                          Teardrop - Massive Attack   
...             ...                                                ...   
393115  zzakkkkkkkk                             Anne - John Frusciante   
393116  zzakkkkkkkk            Passover - 2007 Remaster - Joy Division   
393117  zzakkkkkkkk                             Get the Dutch - Duster   
393118  zzakkkkkkkk                             Hope - John Frusciante   
393119  zzakkkkkkkk                         The Real - John Frusciante   

        Playcount  
0             283  
1             225  
2             187  
3             190  
4          

In [10]:
pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.5 MB/s eta 0:00:00


In [11]:
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares

df_flat['user_id_code'] = df_flat['User'].astype("category").cat.codes
df_flat['item_id_code'] = df_flat['Track_Artist'].astype("category").cat.codes

user_item_matrix = coo_matrix((df_flat['Playcount'].astype(np.float32),
                                (df_flat['user_id_code'], df_flat['item_id_code'])))

user_item_matrix_csr = user_item_matrix.tocsr()

item_user_matrix = user_item_matrix.T.tocsr()

In [12]:
from implicit.evaluation import train_test_split
train_interaction_matrix, test_interaction_matrix = train_test_split(interactions, train_percentage=0.8 )

# AlternatingLeastSquares
The AlternatingLeastSquares model from the implicit library primarily focuses on matrix factorization techniques that work directly with user-item interaction data. It doesn't natively support incorporating user or item features directly into the model during the matrix factorization process in the way that models like LightFM do, which are designed as hybrid recommendation models capable of utilizing both interaction data and metadata (e.g., genre, user demographics).

In [13]:
model_implicit = AlternatingLeastSquares(factors=50, iterations=50, calculate_training_loss=True, regularization=0.05)
model_implicit.fit(train_interaction_matrix)

user_id_map = dict(zip(df_flat['User'].astype("category"), df_flat['user_id_code']))
item_id_map = dict(zip(df_flat['Track_Artist'].astype("category"), df_flat['item_id_code']))

user_code_to_id_map = {v: k for k, v in user_id_map.items()}
item_code_to_id_map = {v: k for k, v in item_id_map.items()}

def recommend_implicit(user_id, model, user_item_matrix_csr, user_id_map, item_code_to_id_map, n_items=10):

    user_code = user_id_map.get(user_id)
    if user_code is None:
        raise ValueError(f"User ID {user_id} not found.")

    recommended, _ = model.recommend(user_code, user_item_matrix_csr[user_code], N=n_items)

    return [item_code_to_id_map.get(item_index, 'Unknown Item') for item_index in recommended]


user_id = df['Username'][0]
recommended_tracks = recommend_implicit(user_id, model_implicit, user_item_matrix_csr, user_id_map, item_code_to_id_map)
print(f"Recommended tracks for user {user_id} using Implicit: {recommended_tracks}")

from implicit.evaluation import precision_at_k, AUC_at_k, ndcg_at_k,mean_average_precision_at_k

# Assuming model, train, and test are already defined
# Calculate precision at k
p_at_k = precision_at_k(model_implicit, train_interaction_matrix, test_interaction_matrix, K=30)

# Calculate AUC
auc_score = AUC_at_k(model_implicit, train_interaction_matrix, test_interaction_matrix)
mean_average_precision_at_k = mean_average_precision_at_k(model_implicit, train_interaction_matrix, test_interaction_matrix)
ndcg_at_k = ndcg_at_k(model_implicit, train_interaction_matrix, test_interaction_matrix)


print(f'Precision at k: {p_at_k}')
print(f'AUC at K: {auc_score}')
print(f'MAP at k: {p_at_k}')
print(f'NDCG at K: {ndcg_at_k}')

from datetime import datetime
current_time = datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H:%M:%S')
import pickle
model_filename = f'/content/drive/MyDrive/Googlecolab/model_implicit_{current_time}.pkl'  # Update the path to your desired Google Drive folder

with open(model_filename, 'wb') as model_file:
    pickle.dump(model_implicit, model_file)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

Recommended tracks for user emosoup using Implicit: ["Alda, Dark Lady Of The Outer Worlds - Julian's Treatment", 'Aleph - Sped Up - You Lost', 'A Buried Sun - In Solitude', 'A Cabin In Montana - Hexvessel', 'Aleister Crowley & Ordo Templi Orientis - Mystifier', 'A Burning Hill - Mitski', 'Change - Blind Melon', 'All the Wine - The National', 'Alcohol-Free - TWICE', 'Alcohol - Barenaked Ladies']


  0%|          | 0/9480 [00:00<?, ?it/s]

  0%|          | 0/9480 [00:00<?, ?it/s]

  0%|          | 0/9480 [00:00<?, ?it/s]

  0%|          | 0/9480 [00:00<?, ?it/s]

Precision at k: 0.10284227889861693
AUC at K: 0.5300421587955412
MAP at k: 0.10284227889861693
NDCG at K: 0.5300421587955412


In [15]:
# Load the model from the file
with open('/content/drive/MyDrive/Googlecolab/model_implicit.pkl', 'rb') as f:
    model = pickle.load(f)

In [26]:
def calculate_diversity_score(recommendations):
    """
    Calculate the diversity score for a list of recommendations based on artist variety.

    :param recommendations: List of recommended 'Track_Artist' strings.
    :return: Diversity score as a float.
    """
    # Extract artists from the recommendations
    artists = set(rec.split(' - ')[-1] for rec in recommendations)

    # Calculate diversity score: number of unique artists divided by the total number of recommendations
    diversity_score = len(artists) / len(recommendations) if recommendations else 0
    return diversity_score

# Example usage
user_id = df['Username'][0]
recommended_tracks = recommend_implicit(user_id, model_implicit, user_item_matrix_csr, user_id_map, item_code_to_id_map)
print(f"Recommended tracks for user {user_id} using Implicit: {recommended_tracks}")
#recommended_tracks = ['Song1 - Artist1', 'Song2 - Artist2', 'Song3 - Artist1', 'Song4 - Artist3', 'Song5 - Artist2']
diversity_score = calculate_diversity_score(recommended_tracks)
print(f"Diversity Score: {diversity_score}")


Recommended tracks for user emosoup using Implicit: ["Alda, Dark Lady Of The Outer Worlds - Julian's Treatment", 'Aleph - Sped Up - You Lost', 'A Buried Sun - In Solitude', 'A Cabin In Montana - Hexvessel', 'Aleister Crowley & Ordo Templi Orientis - Mystifier', 'A Burning Hill - Mitski', 'Change - Blind Melon', 'All the Wine - The National', 'Alcohol-Free - TWICE', 'Alcohol - Barenaked Ladies']
Diversity Score: 1.0
